In [1]:
from collections import OrderedDict
import os

import shap
import lightgbm as lgb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns; sns.set();
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, precision_score, accuracy_score
#import shap
#!pip install shap
from sklearn.manifold import TSNE
#from deep_translator import GoogleTranslator

In [2]:
df = pd.read_csv('votes_all_features.csv')
#df = df.rename({'partido':'party','ano':'year','mes':'month'},axis=1)

In [3]:
# perfil = pd.read_csv('perfil_parlamentar.csv')[['id_parlamentar','UF','genero']].rename({'genero':'gender','UF':'state'},axis=1)
# df = df.merge(perfil, on='id_parlamentar',how='left')

In [4]:
# df = pd.get_dummies(df,columns=['party','state'])#.drop('casa_x_senado',axis=1)

In [5]:
# text = "bom dia amigo"
# translated = GoogleTranslator(source='pt', target='en').translate(text=text)
# translated
# def translate_text(text):
#     return GoogleTranslator(source='pt', target='en').translate(text=text)

In [6]:
# infof = pd.read_csv('info_frentes.csv')[['id_frente','titulo_frente']]
# infof['titulo_frente'] = infof['titulo_frente'].apply(translate_text)

In [7]:
# id2name = dict(zip(infof['id_frente'],infof['titulo_frente']))

In [8]:
# frentes = pd.read_csv('info_membros_frentes.csv')[['id','id_frente']].rename({'id_frente':'front'},axis=1)
# frentes['front'] = frentes['front'].map(id2name)
# frentes = pd.get_dummies(frentes,columns=['front'])
# frentes = frentes.groupby('id').sum().reset_index()
# frentes

In [9]:
# df = df.merge(frentes,left_on='id_parlamentar',right_on='id',how='left')

In [10]:
# inv = pd.read_csv('investimento-partidario.csv')[['id','total_receita_partido','total_receita_candidato','indice_investimento_partido']].rename({'total_receita_partido':'total_party_revenue','total_receita_candidato':'total_candidate_revenue','indice_investimento_partido':'party_investiment_index'},axis=1)
# inv = inv.add_prefix('investiment_')
# inv

In [11]:
# df = df.merge(inv,left_on='id_parlamentar',right_on='investiment_id',how='left')

In [12]:
# df = df.drop(columns=['weight_id','investiment_id'])

In [3]:
lexicons = [feature for feature in df.columns if feature.startswith('Twitter')]
themes = [feature for feature in df.columns if feature.startswith('theme')]
lex_transcricoes = [feature for feature in df.columns if feature.startswith('transcriptions')]
causal_graph = [feature for feature in df.columns if feature.startswith('influence')]
date = ['year','month']
acoes = [feature for feature in df.columns if feature.startswith('actions')]
at_twitter = [feature for feature in df.columns if feature.startswith('twitter-activity')]
autorias = [feature for feature in df.columns if feature.startswith('authorship')]
disc = [feature for feature in df.columns if feature.startswith('discipline')]
peso = [feature for feature in df.columns if feature.startswith('weight')]
doacoes = [feature for feature in df.columns if feature.startswith('donations')]
frentes = [feature for feature in df.columns if feature.startswith('front')]
inv = [feature for feature in df.columns if feature.startswith('investiment')]
partido = [feature for feature in df.columns if feature.startswith('party')]
features = lexicons + themes + lex_transcricoes + causal_graph + date + acoes + at_twitter + autorias + disc + peso + doacoes + frentes + inv + partido

In [4]:
lexicons

['Twitter_Outrage',
 'Twitter_Vagueness',
 'Twitter_Argumentation',
 'Twitter_Modalization',
 'Twitter_Valuation',
 'Twitter_Sentiment',
 'Twitter_Presupposition',
 'Twitter_Environment',
 'Twitter_Economic_Reforms1',
 'Twitter_Economic_Reforms2',
 'Twitter_Economic_Reforms3',
 'Twitter_Armament',
 'Twitter_Racism',
 'Twitter_Covid',
 'Twitter_Impeachment',
 'Twitter_Governism',
 'Twitter_Opposition',
 'Twitter_SentimentAnalisys']

In [14]:
df

,id_votacao,id_parlamentar,voto,data,resumo,id_proposicao,theme_Public Administration,"theme_Agriculture, Livestock, Fisheries and Extractivism","theme_Art, Culture and Religion",theme_Cities and Urban Development,...,front_Parliamentary Front in Defense of Water in the Northeast,front_Parliamentary Front in Defense of the Amazon,front_Parliamentary Front in Defense of the Jequitinhonha River,front_Parliamentary Front in Defense of the São Francisco River,front_Parliamentary Front in Favor of Payment for Environmental Services and Sustainable Production,front_Parliamentary Front of Forestry in Brazil,front_Renewable Energies Joint Parliamentary Front - FPE,investiment_total_party_revenue,investiment_total_candidate_revenue,investiment_party_investiment_index
0,1672576-101,204554,0,2019-03-19,Rejeitado o Requerimento. Sim: 40; não: 317; a...,1672576,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,34500.00,37750.00,0.004558
1,1672576-101,204521,0,2019-03-19,Rejeitado o Requerimento. Sim: 40; não: 317; a...,1672576,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,18474.57,447044.57,0.003906
2,1672576-101,204379,0,2019-03-19,Rejeitado o Requerimento. Sim: 40; não: 317; a...,1672576,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,720316.30,807712.30,0.021495
3,1672576-101,204560,0,2019-03-19,Rejeitado o Requerimento. Sim: 40; não: 317; a...,1672576,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1160000.00,1204707.62,0.014297
4,1672576-101,204528,0,2019-03-19,Rejeitado o Requerimento. Sim: 40; não: 317; a...,1672576,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,817.80,395075.22,0.001279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510287,593065-305,74439,0,2021-03-16,Rejeitada a Emenda nº 9 do Senado Federal. Sim...,593065,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,665000.00,753210.98,0.036907
510288,593065-305,178889,0,2021-03-16,Rejeitada a Emenda nº 9 do Senado Federal. Sim...,593065,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,1.0,0.0,1.0,0.0,885120.00,920076.67,0.007702
510289,593065-305,160632,1,2021-03-16,Rejeitada a Emenda nº 9 do Senado Federal. Sim...,593065,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1103699.68,1774659.84,0.045959
510290,593065-305,204517,1,2021-03-16,Rejeitada a Emenda nº 9 do Senado Federal. Sim...,593065,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,130000.00,232041.66,0.024232


In [15]:
# df.to_csv('votes_all_features.csv',index=False)

In [16]:
# UNWANTED_COLUMNS = ['id_votacao', 'id_parlamentar', 'id_parlamentar_parlametria','id_proposicao','data','obj_votacao', 'is_nominal','resumo','id','ano','mes','id_votacao', 'id_parlamentar', 'voto', 'id_proposicao', 'data', 'obj_votacao', 'casa_y','casa_x', 'is_nominal','id','resumo','partido']
# lexicons = ['Outrage', 'Vagueness', 'Argumentation', 'Modalization', 'Valuation', 'Sentiment', 'Presupposition', 'Ambiental', 'rt_auto', 'ra_auto', 'rp_auto', 'armas_auto',
#             'racismo_auto', 'impeachment_auto', 'governista', 'oposicao','sentimento','transcricoes_Outrage',
#        'transcricoes_Vagueness', 'transcricoes_Argumentation',
#        'transcricoes_Modalization', 'transcricoes_Valuation',
#        'transcricoes_Sentiment', 'transcricoes_Presupposition',
#        'transcricoes_Ambiental', 'transcricoes_rt_auto',
#        'transcricoes_ra_auto', 'transcricoes_rp_auto',
#        'transcricoes_armas_auto', 'transcricoes_racismo_auto',
#        'transcricoes_covid_auto', 'transcricoes_impeachment_auto',
#        'transcricoes_sentimento']

# features = list(set(df.columns.to_list()) - set(UNWANTED_COLUMNS))

target = ['voto']

In [21]:
DEFAULT_LGB_PARAMS = {
    "max_bin": 512,
    "learning_rate": 0.05,
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "num_leaves": 150,
    "verbose": -1,
    "min_data_in_leaf": 5,
    "boost_from_average": True,
    "random_state": 1
}

LABEL_COLUMN_NAME = 'label'
N_FOLDS = 5
N_TIMES_CV = 50

def _robust_evaluation(X, y):
    """N_TIMES_CV times 5-fold cross validation with lgb model
    
    Arguments:
        estimator {[type]} -- [description]
        X {[type]} -- [description]
        y {[type]} -- [description]
    """
    aucs = []
    for seed in range(N_TIMES_CV):
        eval_hist = lgb.cv(DEFAULT_LGB_PARAMS, lgb.Dataset(X, label=y), nfold=N_FOLDS, seed=seed)
        aucs.append(np.mean(eval_hist['auc-mean']))

    score = np.mean(aucs) 

    return score, aucs

def eval_features(df, features, target, random_state):
    """
    5-fold cross validation with LGBMClassifier
    Used to collect results for SHAP analysis
    """
    stats = OrderedDict()
    
    #df = df.sort_values(by = ['data'])
    
    index = df["id_parlamentar"].values.ravel()
    votacoes = df["id_votacao"].values.ravel()
    proposicoes = df["id_proposicao"].values.ravel()
    X = df[features].values
    y = df[target].values.ravel()

    a = [] #array to store AUCs values for each fold
    b = [] #array to store PPVs values for each fold
    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=random_state)
    #cv = TimeSeriesSplit(10)
    max_auc = 0
    iter_max_auc = 0
    for i, (train, val) in enumerate(cv.split(X, y)):
        classifier = lgb.LGBMClassifier(**DEFAULT_LGB_PARAMS)
        classifier = classifier.fit(X[train], y[train])
        
        probas_ = classifier.predict_proba(X[val])
        auc = roc_auc_score(y[val], probas_[:, 1])
        pred_test = classifier.predict(X[val]) #making predictions for test data
        pred_train = classifier.predict(X[train]) #making predictions for train data
        ppv = precision_score(y[val], pred_test) #PPV is also the precision of the positive class
        print(classification_report(y[val],pred_test,output_dict=False))
        a.append(auc)
        b.append(ppv)
        
        if auc > max_auc:
            max_auc = auc
            iter_max_auc = i
        
        
        stats[i] = {}
        stats[i]['model'] = classifier
        stats[i]['auc'] = auc
        stats[i]['X_train'] = X[train]
        stats[i]['y_train'] = y[train]
        stats[i]['y_pred_train'] = pred_train
        stats[i]['X_test'] = X[val]
        stats[i]['y_test'] = y[val]
        stats[i]['y_pred_test'] = pred_test
        stats[i]['index_train'] = index[train]
        stats[i]['index_test'] = index[val]
        stats[i]['votacoes_train'] = votacoes[train]
        stats[i]['votacoes_test'] = votacoes[val]
        stats[i]['proposicoes_train'] = proposicoes[train]
        stats[i]['proposicoes_test'] = proposicoes[val]
        
    stats['mean_auc'] = np.mean(a)
    stats['max_auc'] = np.max(a)
    stats['iter_max_auc'] = iter_max_auc
    stats['mean_ppv'] = np.mean(b)
    return stats, classifier


def meta_learning_eval(df, features, target, random_state):
    stats = OrderedDict()
    
    date = '2021-01-01'
    
    index_train = df["id_parlamentar"][df.data <= date].values.ravel()
    index_test = df["id_parlamentar"][df.data > date].values.ravel()

    votacoes_train = df["id_votacao"][df.data <= date].values.ravel()
    votacoes_test = df["id_votacao"][df.data > date].values.ravel()
   
    proposicoes_train = df["id_proposicao"][df.data <= date].values.ravel()
    proposicoes_test = df["id_proposicao"][df.data > date].values.ravel()
    
    X_train = df[features][df.data <= date].values
    X_test = df[features][df.data > date].values
    
    y_train = df[target][df.data <= date].values.ravel()
    y_test = df[target][df.data > date].values.ravel()

    classifier = lgb.LGBMClassifier(**DEFAULT_LGB_PARAMS)
    classifier = classifier.fit(X_train, y_train)
    
    probas_ = classifier.predict_proba(X_test)
    auc = roc_auc_score(y_test, probas_[:, 1])
    pred_test = classifier.predict(X_test) #making predictions for test data
    pred_train = classifier.predict(X_train) #making predictions for train data
    ppv = precision_score(y_test, pred_test) #PPV is also the precision of the positive class
    print(auc)
    print(classification_report(y_test,pred_test,output_dict=False))
    

In [22]:
stats, classifier = eval_features(df,features,target,1)

              precision    recall  f1-score   support

           0       0.77      0.75      0.76     47324
           1       0.79      0.81      0.80     54735

    accuracy                           0.78    102059
   macro avg       0.78      0.78      0.78    102059
weighted avg       0.78      0.78      0.78    102059

[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
              precision    recall  f1-score   support

           0       0.77      0.75      0.76     47324
           1       0.79      0.81      0.80     54735

    accuracy                           0.78    102059
   macro avg       0.78      0.78      0.78    102059
weighted avg       0.78      0.78      0.78    102059

[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
              precision    recall  f1-score   support

           0       0.77      0.75      0.76     473

In [24]:
stats['max_auc']

0.8739861191250228

In [18]:
DEFAULT_LGB_PARAMS = {
    "max_bin": 512,
    "learning_rate": 0.05,
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "num_leaves": 150,
    "verbose": -1,
    "min_data_in_leaf": 5,
    "boost_from_average": True,
    "random_state": 1
}

def strat_meta_learning(df, features, target, random_state):
    stats = OrderedDict()
        
    index = df["id_parlamentar"].values.ravel()
    votacoes = df["id_votacao"].values.ravel()
    proposicoes = df["id_proposicao"].values.ravel()
    X = df[features].values
    y = df[target].values.ravel()

    a = [] #array to store AUCs values for each fold
    b = [] #array to store PPVs values for each fold
    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=random_state)
    max_auc = 0
    iter_max_auc = 0
    for i, (train, val) in enumerate(cv.split(X, y)):
        classifier = lgb.LGBMClassifier(**DEFAULT_LGB_PARAMS)
        classifier = classifier.fit(X[train], y[train])
        
        probas_ = classifier.predict_proba(X[val])
        auc = roc_auc_score(y[val], probas_[:, 1])
        pred_test = classifier.predict(X[val]) #making predictions for test data
        pred_train = classifier.predict(X[train]) #making predictions for train data
        ppv = precision_score(y[val], pred_test) #PPV is also the precision of the positive class
        #print(classification_report(y[val],pred_test,output_dict=False))
        a.append(auc)
        b.append(ppv)
        
        if auc > max_auc:
            max_auc = auc
            iter_max_auc = i
        
        stats[i] = {}
        stats[i]['model'] = classifier
        stats[i]['auc'] = auc
        stats[i]['X_train'] = X[train]
        stats[i]['y_train'] = y[train]
        stats[i]['y_pred_train'] = pred_train
        stats[i]['X_test'] = X[val]
        stats[i]['y_test'] = y[val]
        stats[i]['y_pred_test'] = pred_test
        stats[i]['index_train'] = index[train]
        stats[i]['index_test'] = index[val]
        stats[i]['votacoes_train'] = votacoes[train]
        stats[i]['votacoes_test'] = votacoes[val]
        stats[i]['proposicoes_train'] = proposicoes[train]
        stats[i]['proposicoes_test'] = proposicoes[val]
        
    stats['mean_auc'] = np.mean(a)
    stats['max_auc'] = np.max(a)
    stats['iter_max_auc'] = iter_max_auc
    stats['mean_ppv'] = np.mean(b)
    
    print(stats['max_auc'])
    
    selected_iter = stats['iter_max_auc']
    X = stats[selected_iter]['X_train']
    X_test = stats[selected_iter]['X_test']
    y = stats[selected_iter]['y_train']
    y_test = stats[selected_iter]['y_test']
    indexes = stats[selected_iter]['index_train']
    votacoes = stats[selected_iter]['votacoes_train']
    proposicoes = stats[selected_iter]['proposicoes_train']
    y_pred = stats[selected_iter]['y_pred_train']
    y_pred_test = stats[selected_iter]['y_pred_test']
    
    model = stats[selected_iter]['model']
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)[1]
    shap_values_test = explainer.shap_values(X_test)[1]
    
    classifier = lgb.LGBMClassifier(**DEFAULT_LGB_PARAMS)
    classifier = classifier.fit(shap_values, y)
    
    probas_ = classifier.predict_proba(shap_values_test)
    auc = roc_auc_score(y_test, probas_[:, 1])
    
    print('max_auc:',max_auc)
    print('meta_auc:',auc)
    
    joint_train = np.concatenate((X,shap_values),axis=1)
    joint_test = np.concatenate((X_test,shap_values_test),axis=1)
    classifier = lgb.LGBMClassifier(**DEFAULT_LGB_PARAMS)
    classifier = classifier.fit(joint_train, y)
    
    probas_ = classifier.predict_proba(joint_test)
    auc = roc_auc_score(y_test, probas_[:, 1])
    print('meta+features_auc:',auc)
    

In [19]:
stats, classifier = strat_meta_learning(df,features,target,1)

[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
0.856932602280613


LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
max_auc: 0.856932602280613
meta_auc: 0.8593278903891578
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
meta+features_auc: 0.859033942397993


TypeError: cannot unpack non-iterable NoneType object

In [15]:
a1 = np.zeros((2,3))
a2 = np.zeros((2,5))
np.concatenate((a1, a2), axis=1)

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])